In [1]:
#!/usr/bin/env python3
# -- coding: utf-8 --
"""
Ce script télécharge les cours boursiers historiques (prix ajustés) pour une liste de tickers
issue du dossier "intraQuarter/_KeyStats/". Il enregistre ensuite ces données dans des CSV.
Il télécharge également l'ETF SPY (S&P500) sur la même période.
"""
!pip install yfinance
!pip install pandas_datareader
!pip install pandas


import os
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

# On redéfinit la fonction de téléchargement pour utiliser yfinance
pdr.get_data_yahoo = yf.download

# Définition de la période de téléchargement
START_DATE = "2018-01-01"
END_DATE = "2023-01-01"

In [2]:

def build_stock_dataset(start=START_DATE, end=END_DATE):
    """
    Télécharge et construit un CSV stock_prices.csv contenant
    le prix de clôture ajusté (Adj Close) pour tous les tickers
    trouvés dans le dossier intraQuarter/_KeyStats/.
    """

    # Chemin menant au dossier contenant les tickers
    statspath = "intraQuarter/_KeyStats/"

    # On récupère la liste des fichiers et dossiers à l'intérieur de statspath
    ticker_list = os.listdir(statspath)

    # Sur macOS, ".DS_Store" est un fichier caché qui peut gêner la liste des tickers
    # On le retire s'il existe
    if ".DS_Store" in ticker_list:
        os.remove(os.path.join(statspath, ".DS_Store"))
        ticker_list.remove(".DS_Store")

    # (Optionnel) S'assurer que tout est en majuscules (par sécurité)
    ticker_list = [t.upper() for t in ticker_list]

    # Téléchargement des données pour tous les tickers, en désactivant l'ajustement automatique
    # pour avoir la colonne "Adj Close" séparée de "Close"
    all_data = pdr.get_data_yahoo(ticker_list, start=start, end=end, auto_adjust=False)

    # On ne garde que la colonne "Adj Close"
    stock_data = all_data["Adj Close"]

    # Certains tickers peuvent être vides (aucune donnée) : on les retire
    stock_data = stock_data.copy()
    stock_data.dropna(how="all", axis=1, inplace=True)
    stock_data.ffill(inplace=True)


    # Pour information : on liste les tickers manquants
    # (i.e. ceux qui ne figurent pas en colonne)
    missing_tickers = [
        ticker for ticker in ticker_list
        if ticker not in stock_data.columns
    ]
    print(f"{len(missing_tickers)} tickers manquants : {missing_tickers}")

    # On applique un forward fill pour combler les dates manquantes (si besoin)
    stock_data.ffill(inplace=True)

    # On enregistre le DataFrame final dans un CSV
    stock_data.to_csv("stock_prices.csv")
    print("Fichier stock_prices.csv généré.")

In [3]:
def build_sp500_dataset(start=START_DATE, end=END_DATE):
    """
    Télécharge et enregistre l'ETF SPY (qui suit l'indice S&P500)
    dans un CSV sp500_index.csv.
    """
    # Même logique, on peut choisir auto_adjust=False ou True
    # Ici, peu importe, puisque c'est un seul ticker
    index_data = pdr.get_data_yahoo("SPY", start=START_DATE, end=END_DATE)
    index_data.reset_index(inplace=True)  # remet la colonne Date au lieu de l'index
    index_data.to_csv("sp500_index.csv", index=False)

    print("Fichier sp500_index.csv généré.")

In [4]:
def build_dataset_iteratively(idx_start, idx_end, date_start=START_DATE, date_end=END_DATE):
    """
    Télécharge les données ticker par ticker (utile si la liste est trop grande
    et que la requête groupée échoue).
    - idx_start, idx_end : indices pour récupérer une partie (un "chunk") de la liste de tickers.
    - date_start, date_end : limites temporelles du téléchargement.
    """
    statspath = "intraQuarter/_KeyStats/"
    ticker_list = os.listdir(statspath)

    # On retire .DS_Store s'il existe
    if ".DS_Store" in ticker_list:
        os.remove(os.path.join(statspath, ".DS_Store"))
        ticker_list.remove(".DS_Store")

    # On crée un DataFrame vide pour y concaténer les prix ajustés de chaque ticker
    df = pd.DataFrame()

    # On parcourt les tickers sur la portion [idx_start:idx_end]
    for ticker in ticker_list[idx_start:idx_end]:
        ticker = ticker.upper()
        try:
            # Téléchargement des données pour 1 ticker
            stock_ohlc = pdr.get_data_yahoo(ticker, start=date_start, end=date_end, auto_adjust=False)
            if stock_ohlc.empty:
                print(f"Aucune donnée pour {ticker}")
                continue

            # On renomme la colonne 'Adj Close' pour la concaténer plus facilement
            adj_close = stock_ohlc["Adj Close"].rename(ticker)
            df = pd.concat([df, adj_close], axis=1)

        except Exception as e:
            print(f"Erreur lors du téléchargement de {ticker}: {e}")

    # On enregistre le résultat dans un CSV
    df.to_csv("stock_prices.csv", mode="w")
    print("Fichier stock_prices.csv généré (mode itératif).")

In [5]:
if __name__ == "__main__":
    build_stock_dataset()
    build_sp500_dataset()


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed

0 tickers manquants : []
Fichier stock_prices.csv généré.
YF.download() has changed argument auto_adjust default to True
Fichier sp500_index.csv généré.
